In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


# from langchain.vectorstores import Pinecone
import pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.vectorstores import Pinecone as PineconeStore

c:\Users\akamma1\Videos\Langchain\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY = "9f3cbd34-0ab3-41b6-bf1f-ead705825563"
PINECONE_API_ENV = "us-east-1"


def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# Download the embeddings
embeddings = download_hugging_face_embeddings()
pc = Pinecone(api_key=PINECONE_API_KEY)
spec = ServerlessSpec(cloud = "aws", region=PINECONE_API_ENV)

In [4]:
import time

index_name = "medical-chatbot"

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,  # dimensionality of embed 3
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 7020}},
 'total_vector_count': 7020}

In [5]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

#vectordb = PineconeStore(index, embeddings,"text")



In [6]:
# Define a function to get query embeddings
def embed_query(query):
    return embeddings.embed_documents([query])[0]

# Pass the embedding function correctly
vectorstore = PineconeStore(index, embed_query, "text")

# Example query to test
query = "Example query"
query_embedding = vectorstore._embedding_function(query)

In [7]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
    chain_type="stuff",
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True
)

In [8]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Response :  The blood typing and crossmatching tests are based on the presence or absence of specific antigens on the surface of red blood cells. These antigens are proteins that can trigger an immune response if they are recognized as foreign by the body. The most common blood types are determined by the presence or absence of these antigens, which are inherited from an individual's parents.
